<a href="https://colab.research.google.com/github/eder1985/pismo_recruiting_technical_case/blob/main/work/notebooks/Colab_Pismo_Recruiting_Technical_Case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center>Pismo Recruiting Technical Case</center></h1>

---



## 1. Pre-requisites

### Installing Spark

Install Dependencies:


1.   Java 8
2.   Apache Spark with hadoop and
3.   Findspark (used to locate the spark in the system)


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

Set Environment Variables:

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
!ls

sample_data  spark-3.1.1-bin-hadoop3.2	spark-3.1.1-bin-hadoop3.2.tgz


In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [ ]:
!python --version

Python 3.10.6


### Creating data folders

In [ ]:
!mkdir -p work/data/raw/events/
!mkdir -p work/data/processed/events/
!mkdir -p work/data/trusted/events/

## 2. Generate Fake Data

### Installing libs

In [ ]:
!pip install -q faker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.2 MB/s eta 0:00:00


### Imports

In [ ]:
from faker import Faker
from faker.providers import BaseProvider
from datetime import datetime
from json import dumps
import pandas as pd
import random
import collections
import glob
import os

### Generating fake `event_id`: random UUIDs for unique and non-repeated event_id values

In [ ]:
fake = Faker()
Faker.seed(random.randrange(0, 99999999999999999999, 1))
fake_event_id = fake.uuid4()
print(fake_event_id)

32e7615a-0d59-4494-a175-4b93f2082a64


### Generating custom fake `event_id`: random UUIDs for duplicate event_id scenario

In [ ]:
class CustomUUIDProvider(BaseProvider):
    def custom_uuid(self):
        list_uuids = [
            '1a1a1a1a-1a1a-1a1a-1a1a-1a1a1a1a1a1a',
            '2b2b2b2b-2b2b-2b2b-2b2b-2b2b2b2b2b2b',
            '3c3c3c3c-3c3c-3c3c-3c3c-3c3c3c3c3c3c'
            ]
        return random.choice(list_uuids)

### Generating fake `timestamp`: random timestamps with values until 3 years ago

In [ ]:
fake_timestamp = datetime.strftime(fake.date_time_between(start_date='-3y', end_date='now'),"%Y-%m-%dT%H:%M:%S")
print(fake_timestamp)

2022-12-22T08:53:28


### Generating custom fake `event_type`: random values based on list


In [ ]:
class EventTypeProvider(BaseProvider):
    def event_type(self):
        list_event_types = ['account-status-change','transaction-new-value']
        return random.choice(list_event_types)

fake.add_provider(EventTypeProvider)

fake_event_type = fake.event_type()
print(fake_event_type)

account-status-change


### Generating fake `status`: random values based on list

In [ ]:
class StatusTypeProvider(BaseProvider):
    def status_type(self):
        list_status_types = ['ACTIVE','INACTIVE','SUSPENDED','BLOCKED', 'DELETED']
        return random.choice(list_status_types)

fake.add_provider(StatusTypeProvider)

fake_status_type = fake.status_type()
print(fake_status_type)

ACTIVE


### Generating custom fake `data`: values based on dict

In [49]:
def custom_data(fake):
  dict_data = {
      "account-status-change": collections.OrderedDict([
          ('id', fake.random_number(digits=6)),
          ('old_status', fake.status_type()),
          ('new_status', fake.status_type()),
          ('reason', fake.sentence(nb_words=5))
      ]),
      "transaction-new-value": collections.OrderedDict([
          ('id', fake.random_number(digits=6)),
          ('account_orig_id', fake.random_number(digits=6)),
          ('account_dest_id', fake.random_number(digits=6)),
          ('amount', fake.pyfloat(positive=True)),
          ('currency', fake.currency_code())
      ])
  }
  return dict_data

fake_custom_data = custom_data(fake).get(fake.event_type())
print(fake_custom_data)

OrderedDict([('id', 730881), ('account_orig_id', 438364), ('account_dest_id', 643908), ('amount', 70.9270861860859), ('currency', 'ALL')])


### Defining `write_fake_data` and `read_fake_data` functions

In [51]:
def write_fake_data(fake, length, destination_path, unique_uuid = True):

    database = []
    current_time = datetime.now().strftime("%Y%m%d%H%M%S")
    filename = 'fake_events_'+current_time

    for x in range(length):
        uuid = fake.uuid4() if unique_uuid else fake.custom_uuid()
        event_type = fake.event_type()
        project_domain_name = event_type.split('-')[0]

        database.append(collections.OrderedDict([
            ('event_id', uuid),
            ('timestamp', datetime.strftime(fake.date_time_between(start_date='-3y', end_date='now'),"%Y-%m-%dT%H:%M:%S")),
            ('domain', project_domain_name),
            ('event_type', event_type),
            ('data', custom_data(fake).get(event_type))
        ]))

    with open('%s%s.json' % (destination_path, filename), 'w') as output:
        output.write(dumps(database, indent=4, sort_keys=False, default=str))

    print("Done.")

def read_fake_data(json_filepath):
    json_files = [os.path.normpath(i) for i in glob.glob(json_filepath)]
    df = pd.concat([pd.read_json(f) for f in json_files])
    return df

def run(length, unique_uuid = True):
    fake = Faker()
    Faker.seed(random.randrange(0, 99999999999999999999, 1))
    fake.add_provider(StatusTypeProvider)
    fake.add_provider(CustomUUIDProvider)
    fake.add_provider(EventTypeProvider)

    destination_path = 'work/data/raw/events/'
    write_fake_data(fake, length, destination_path,unique_uuid)

    json_filepath = destination_path+'*.json'
    fake_data = read_fake_data(json_filepath)
    print(fake_data)

### Writing and reading fake data

> Writing json file with 1000 unique events

In [52]:
run(1000)

Done.
                                 event_id           timestamp       domain  \
0    1d85a4de-7c6d-441c-bffa-4aa02d9cd2db 2022-12-02 20:18:05  transaction   
1    5994b3e3-ab7b-42dc-93ff-a56e128d8ce1 2020-07-29 08:10:32  transaction   
2    b4408e83-d13e-440a-b541-16f039de9058 2022-02-17 17:25:17      account   
3    040173db-2846-404f-9097-67cb6276fbc2 2021-08-15 22:45:54      account   
4    28c59442-8ada-475e-914a-5f5bdf57d84f 2020-08-20 10:16:08      account   
..                                    ...                 ...          ...   
995  8743107b-89dd-4e6f-8427-70edfee3cbe4 2021-01-10 04:51:35  transaction   
996  0a7c6232-ef1d-4595-bc6e-c173c13c7902 2021-09-12 20:53:08      account   
997  f38d7e3b-8aed-483e-a886-16701f806d6e 2023-04-06 08:42:01  transaction   
998  9cc1094d-fa65-4b95-b861-084a914ed200 2021-12-04 08:03:29      account   
999  1479f643-586b-417d-bf40-e7f972625fcf 2022-09-11 21:55:39      account   

                event_type                               

> Writing json file with 100 events with duplicate rows

In [55]:
run(100,unique_uuid = False)

Done.
                                event_id           timestamp       domain  \
0   1d85a4de-7c6d-441c-bffa-4aa02d9cd2db 2022-12-02 20:18:05  transaction   
1   5994b3e3-ab7b-42dc-93ff-a56e128d8ce1 2020-07-29 08:10:32  transaction   
2   b4408e83-d13e-440a-b541-16f039de9058 2022-02-17 17:25:17      account   
3   040173db-2846-404f-9097-67cb6276fbc2 2021-08-15 22:45:54      account   
4   28c59442-8ada-475e-914a-5f5bdf57d84f 2020-08-20 10:16:08      account   
..                                   ...                 ...          ...   
95  3c3c3c3c-3c3c-3c3c-3c3c-3c3c3c3c3c3c 2021-10-05 00:34:18  transaction   
96  1a1a1a1a-1a1a-1a1a-1a1a-1a1a1a1a1a1a 2021-08-26 20:34:46  transaction   
97  2b2b2b2b-2b2b-2b2b-2b2b-2b2b2b2b2b2b 2022-01-31 07:10:49  transaction   
98  1a1a1a1a-1a1a-1a1a-1a1a-1a1a1a1a1a1a 2021-02-18 21:52:26      account   
99  3c3c3c3c-3c3c-3c3c-3c3c-3c3c3c3c3c3c 2021-12-01 00:03:34      account   

               event_type                                            

## 3. Exploring the Raw Dataframe

### Loading the Dataframe:

- List raw json files
- Define raw_events_schema with only StringType for not lost data
- Load raw_events dataframe

In [ ]:
!ls work/data/raw/events/ -la

total 428
drwxr-xr-x 2 root root   4096 Jul 27 16:30 .
drwxr-xr-x 3 root root   4096 Jul 27 16:28 ..
-rw-r--r-- 1 root root 387821 Jul 27 16:28 fake_events_20230727162819.json
-rw-r--r-- 1 root root  38965 Jul 27 16:30 fake_events_20230727163006.json


In [56]:

from pyspark.sql.types import StructType,StructField, StringType

raw_events_schema = StructType([
      StructField("data",StringType(),True),
      StructField("domain",StringType(),True),
      StructField("event_id",StringType(),True),
      StructField("event_type",StringType(),True),
      StructField("timestamp",StringType(),True)
  ])

In [57]:
raw_events = spark.read.option("multiline","true").schema(raw_events_schema).json('work/data/raw/events/')
raw_events.show(5, truncate = False)

+----------------------------------------------------------------------------------------------------------------+-----------+------------------------------------+---------------------+-------------------+
|data                                                                                                            |domain     |event_id                            |event_type           |timestamp          |
+----------------------------------------------------------------------------------------------------------------+-----------+------------------------------------+---------------------+-------------------+
|{"id":833589,"account_orig_id":808931,"account_dest_id":676654,"amount":9.36571589141128E9,"currency":"CHF"}    |transaction|1d85a4de-7c6d-441c-bffa-4aa02d9cd2db|transaction-new-value|2022-12-02T20:18:05|
|{"id":622108,"account_orig_id":866563,"account_dest_id":255941,"amount":8.9003365575304E10,"currency":"MWK"}    |transaction|5994b3e3-ab7b-42dc-93ff-a56e128d8ce1|transaction-n

In [58]:
raw_events.count()

1100

### Dataframe Raw Schema

In [59]:
raw_events.printSchema()

root
 |-- data: string (nullable = true)
 |-- domain: string (nullable = true)
 |-- event_id: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- timestamp: string (nullable = true)



## 4. Applying transformations

### Columns `timestamp` and `day` transformations

In [ ]:
partial_events = raw_events\
  .withColumn("timestamp",to_timestamp("timestamp"))\
  .withColumn("day",to_date("timestamp"))

In [ ]:
partial_events.printSchema()

root
 |-- data: string (nullable = true)
 |-- domain: string (nullable = true)
 |-- event_id: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- day: date (nullable = true)



### Drop duplicated events

In [ ]:
from pyspark.sql.functions import countDistinct

# Count
partial_events.select(countDistinct("event_id", "event_type").alias("distinct_events")).show()


+---------------+
|distinct_events|
+---------------+
|           1006|
+---------------+



In [ ]:
grouped_events = partial_events \
  .groupBy( \
      col("event_id"), \
      col("event_type")) \
  .agg( \
      max(col("timestamp")))

grouped_events.show(truncate = False)

+------------------------------------+---------------------+-------------------+
|event_id                            |event_type           |max(timestamp)     |
+------------------------------------+---------------------+-------------------+
|c8b7a073-2967-42a2-b1a2-57932bac2d80|transaction-new-value|2023-01-12 09:34:38|
|d5ee70c5-72e2-4dfc-b044-8a02b6495af9|transaction-new-value|2023-04-19 07:46:50|
|52e834f4-1569-440c-acdd-970ab5a268ad|account-status-change|2023-03-21 08:23:33|
|a3115854-83f0-4dca-a592-dcd99541631e|account-status-change|2023-03-01 03:13:25|
|c0226b43-9864-4fdb-8060-53d465cf5108|transaction-new-value|2022-11-18 00:20:19|
|5e04f614-6c0a-49ba-8863-276a955b4256|transaction-new-value|2020-09-29 02:48:30|
|1f6c4ab7-27ec-4f11-a0ee-ebd4ce14ea6b|transaction-new-value|2020-10-04 18:12:24|
|c5e337bc-9f41-4d69-af2b-d0552e6c0011|transaction-new-value|2022-12-12 21:27:54|
|1447f2d2-7a1b-469f-942a-aa3589e26a5b|account-status-change|2023-04-22 18:32:47|
|d0590eb9-c506-454e-9705-227

In [ ]:
final_events = grouped_events \
    .join(partial_events, ["event_id","event_type"]) \
    .dropDuplicates(["event_id","event_type"]) \
    .drop("timestamp") \
    .withColumnRenamed("max(timestamp)", "timestamp")
final_events.show()

+--------------------+--------------------+-------------------+--------------------+-----------+----------+
|            event_id|          event_type|          timestamp|                data|     domain|       day|
+--------------------+--------------------+-------------------+--------------------+-----------+----------+
|52e834f4-1569-440...|account-status-ch...|2023-03-21 08:23:33|{"id":625198,"old...|    account|2023-03-21|
|a3115854-83f0-4dc...|account-status-ch...|2023-03-01 03:13:25|{"id":132643,"old...|    account|2023-03-01|
|c0226b43-9864-4fd...|transaction-new-v...|2022-11-18 00:20:19|{"id":398056,"acc...|transaction|2022-11-18|
|c8b7a073-2967-42a...|transaction-new-v...|2023-01-12 09:34:38|{"id":31953,"acco...|transaction|2023-01-12|
|d5ee70c5-72e2-4df...|transaction-new-v...|2023-04-19 07:46:50|{"id":494669,"acc...|transaction|2023-04-19|
|1447f2d2-7a1b-469...|account-status-ch...|2023-04-22 18:32:47|{"id":958560,"old...|    account|2023-04-22|
|1f6c4ab7-27ec-4f1...|transa

In [ ]:
final_events.count()

1006

### Flatten `data` column by event-type

> Each event-type has its own schema

In [ ]:
final_events.groupBy('event_type').agg(count(col('event_type'))).show(truncate=False)

+---------------------+-----------------+
|event_type           |count(event_type)|
+---------------------+-----------------+
|account-status-change|483              |
|transaction-new-value|523              |
+---------------------+-----------------+



In [ ]:
def distinct_values(df, col):
    list_distinct_values = df.select(col).distinct().toPandas()[col].to_list()
    return list_distinct_values

def flatten_df(df):
    json_schema = spark.read.json(df.rdd.map(lambda row: row.data)).schema
    df2 = df.withColumn("data", from_json("data", json_schema))
    col1 = df2.columns
    col1.remove("data")
    col2 = df2.select("data.*").columns
    append_str ="data."
    col3 = [append_str + val for val in col2]
    col_list = col1 + col3
    df3 = df2.select(*col_list).drop("data")
    return df3

def write_parquet_data(df):
    df.printSchema()
    df.write \
    .partitionBy("event_type", "day") \
    .mode("append") \
    .parquet("work/data/trusted/events/")

## 5. Write transformed data in parquet format





In [ ]:
for value in distinct_values(final_events, 'event_type'):
    filtered_df = final_events.filter(col('event_type') == value)
    flattened_df = flatten_df(filtered_df)
    write_parquet_data(flattened_df)

root
 |-- event_id: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- domain: string (nullable = true)
 |-- day: date (nullable = true)
 |-- id: long (nullable = true)
 |-- new_status: string (nullable = true)
 |-- old_status: string (nullable = true)
 |-- reason: string (nullable = true)

root
 |-- event_id: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- domain: string (nullable = true)
 |-- day: date (nullable = true)
 |-- account_dest_id: long (nullable = true)
 |-- account_orig_id: long (nullable = true)
 |-- amount: double (nullable = true)
 |-- currency: string (nullable = true)
 |-- id: long (nullable = true)



## 6. Read transformed data in parquet format

In [ ]:
!ls -la work/data/trusted/events/

total 52
drwxr-xr-x   4 root root  4096 Jul 27 17:58  .
drwxr-xr-x   3 root root  4096 Jul 27 16:28  ..
drwxr-xr-x 392 root root 20480 Jul 27 17:57 'event_type=account-status-change'
drwxr-xr-x 406 root root 20480 Jul 27 17:58 'event_type=transaction-new-value'
-rw-r--r--   1 root root     0 Jul 27 17:58  _SUCCESS
-rw-r--r--   1 root root     8 Jul 27 17:58  ._SUCCESS.crc


In [ ]:
trusted_transaction_events = spark.read.parquet('work/data/trusted/events/event_type=transaction-new-value')
trusted_transaction_events.show()

+--------------------+-------------------+-----------+---------------+---------------+-------------------+--------+------+----------+
|            event_id|          timestamp|     domain|account_dest_id|account_orig_id|             amount|currency|    id|       day|
+--------------------+-------------------+-----------+---------------+---------------+-------------------+--------+------+----------+
|5e04f614-6c0a-49b...|2020-09-29 02:48:30|transaction|         263662|         759139| 8.17173960051861E9|     AED|237285|2020-09-29|
|7312e48d-158b-432...|2020-09-29 17:38:56|transaction|         885368|         727721| 3.41947704352005E9|     PGK| 57630|2020-09-29|
|e1d73bb0-2a74-450...|2020-09-29 17:31:25|transaction|         534542|         694999|   5332419.77476907|     ZAR|346215|2020-09-29|
|dc63e448-4908-453...|2022-07-16 23:57:53|transaction|          57176|         786839|   138531.541359864|     RWF|370925|2022-07-16|
|6331dd00-a620-470...|2021-10-22 01:24:00|transaction|        

In [ ]:
trusted_transaction_events.count()

523

In [ ]:
trusted_account_events = spark.read.parquet('work/data/trusted/events/event_type=account-status-change')
trusted_account_events.show()

+--------------------+-------------------+-------+------+----------+----------+--------------------+----------+
|            event_id|          timestamp| domain|    id|new_status|old_status|              reason|       day|
+--------------------+-------------------+-------+------+----------+----------+--------------------+----------+
|b7c638b2-5cb1-440...|2021-01-01 00:10:00|account|989694|   BLOCKED| SUSPENDED|Former small poli...|2021-01-01|
|2b58a0d5-459b-434...|2021-01-23 18:28:00|account|145771|   BLOCKED|    ACTIVE|Both relationship...|2021-01-23|
|397ce6c9-8d98-486...|2022-10-18 17:57:53|account|540100| SUSPENDED|   BLOCKED|Exist agreement m...|2022-10-18|
|251b3d2b-1e75-4cd...|2020-11-27 07:26:53|account|917054|   DELETED| SUSPENDED|Computer garden e...|2020-11-27|
|ae1a431e-3c58-498...|2022-12-14 13:21:13|account|874415| SUSPENDED|  INACTIVE|Officer often fou...|2022-12-14|
|fca5219e-8933-43f...|2023-07-05 10:41:27|account|789096|  INACTIVE|  INACTIVE|Specific product ...|2023

In [ ]:
trusted_account_events.count()

483

## 7. Move raw data to processed data folder

In [ ]:
!mv work/data/raw/events/* work/data/processed/events/
!ls work/data/processed/events/ -la

total 428
drwxr-xr-x 2 root root   4096 Jul 27 17:59 .
drwxr-xr-x 3 root root   4096 Jul 27 16:28 ..
-rw-r--r-- 1 root root 387821 Jul 27 16:28 fake_events_20230727162819.json
-rw-r--r-- 1 root root  38965 Jul 27 16:30 fake_events_20230727163006.json
